In [237]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [238]:
import nltk
import io
import random
import string
import json
import numpy as np
import pandas as pd
import datetime as dt
import itertools
import warnings

from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize , sent_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from math import log10

warnings.filterwarnings('ignore')

## Download all necessay NLTK corpus

In [239]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_da

True

## Import Question *Dataset
**\*obtained from HAI COMP3074 Lab**

In [240]:
file = pd.read_csv("COMP3074-CW1-Dataset.csv")
file.drop(file.columns[[0,1, 3]], axis=1,inplace=True)
file.columns = file.loc[0]
file = file.drop(0)
file.head()
file = file.to_string(index=False)

## Question Dataset Preprocessing
(activities : tokenizing, lowercase, pos-tag, lemmatization, stopword removal)<br>
**code referenced from HAI COMP3074 Lab sessions**

In [241]:
# tokenize
tokenizer = nltk.RegexpTokenizer(r"\w+")
sentence_tokens = nltk.sent_tokenize(str(file))
sent_tokens = []

for sentence in sentence_tokens:
  tok_text = tokenizer.tokenize(sentence)

  # set lowercase
  lowered_tok_text = [word.lower() for word in tok_text]

  # Pos-tagging
  lowered_tok_text = nltk.pos_tag(lowered_tok_text , tagset ='universal')

  # mapping for wordnet
  def nltk_pos_tagger(nltk_tag):
      if nltk_tag.startswith('J'):
          return wordnet.ADJ
      elif nltk_tag.startswith('V'):
          return wordnet.VERB
      elif nltk_tag.startswith('N'):
          return wordnet.NOUN
      elif nltk_tag.startswith('R'):
          return wordnet.ADV
      else:          
          return None

  wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), lowered_tok_text)

  # lemmetize text
  lemmatiser = WordNetLemmatizer ()
  lemmatized_sentence = []
      
  for word, tag in wordnet_tagged:
      if tag is None:
          lemmatized_sentence.append(word)
      else:        
          lemmatized_sentence.append(lemmatiser.lemmatize(word, tag))

  # remove stopwords
  english_stopwords = stopwords.words('english')
  lemmatized_sentence = [word for word in lemmatized_sentence if word not in english_stopwords]
  lemmatized_sentence = ' '.join(str(word) for word in lemmatized_sentence)
  sent_tokens.append(lemmatized_sentence)

## Visualization of Question Dataset
**code referenced from HAI COMP3074 Lab sessions**

In [242]:
# create chatbot vocabulary
vocabulary = set(sent_tokens)
vocabulary = list(sent_tokens)

In [243]:
# Create Bag-of-words model
bag_of_words = {}
bag_of_words = np.zeros(len(vocabulary))
for word in sent_tokens:
  index = vocabulary.index(word)
  bag_of_words[index] += 1
print(f'bag-of-word: {bag_of_words[500:510]}')

bag-of-word: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [244]:
matrix = np.vstack(bag_of_words)
print(f"Document-term matrix {matrix.shape}:\n{matrix}")
print(f"Term-document matrix {matrix.transpose().shape}:\n{matrix.transpose()}")

Document-term matrix (1501, 1):
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [

In [245]:
#save matrix
print(matrix)
np.save('./dataset_matrix.npy', matrix)  # (uses pickle)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.

In [246]:
#load matrix
loaded_matrix = np.load('./dataset_matrix.npy')
print(loaded_matrix)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.

In [247]:
def logfreq_weighting(vector):
    lf_vector = []
    for frequency in vector:
        lf_vector.append(log10(1+frequency))
    return np.array(lf_vector)

In [248]:
def sim_cosine(vector_1, vector_2):
    similarity = 1 - spatial.distance.cosine(vector_1, vector_2)
    return similarity

In [249]:
logfreq_bag_of_words = {}
logfreq_bag_of_words = logfreq_weighting(bag_of_words)
print(f'bag-of-word (logfreq weighted): {logfreq_bag_of_words}')

bag-of-word (logfreq weighted): [0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.47712125 0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.47712125 0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103    0.30103    0.30103    0.30103    0.30103    0.30103
 0.30103

## Import Intents Dataset

In [250]:
intents = json.loads(open('intents.json').read())

documents = []
words = []
classes = []

for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list)
		documents.append((word_list, intent['tag']))
		if intent['tag'] not in classes:
			classes.append(intent['tag'])
   
## Unused general bag-of-words for intents json file ##
words = ' '.join(str(word) for word in words)
words = tokenizer.tokenize(words)
words = [word for word in words if word not in english_stopwords]
words = [lemmatiser.lemmatize(word) for word in words]
words = sorted(set(words))

intents_bag_of_words = {}
intents_bag_of_words = np.zeros(len(words))
for word in words:
  index = words.index(word)
  intents_bag_of_words[index] += 1

## Build Intents Bag-of-words Model
**Idea and code referenced from 'https://www.christianhaller.me/blog/projectblog/2020-07-07-How-to-train-your-NLP-Chatbot/' and COMP3074 Lab Sessions respectively**

In [251]:
intent_corpus = {}

for pair in documents:
  sentence = []

  for word in pair[0]:
    sentence.append(word)

  #sentence = ' '.join(str(word) for word in sentence)

  # POS-tagging
  filtered_query = nltk.pos_tag(sentence , tagset ='universal')

  wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), filtered_query)

  # lemmetize text
  sentence = []
      
  for word, tag in wordnet_tagged:
      if tag is None:
          sentence.append(word)
      else:        
          sentence.append(lemmatiser.lemmatize(word, tag))

  sentence = ' '.join(str(word) for word in sentence)
  sentence = tokenizer.tokenize(sentence)
  sentence = [word for word in sentence if word not in english_stopwords]
  sentence = [lemmatiser.lemmatize(word) for word in sentence]
  sentence = list(sorted(set(sentence)))

  if pair[1] not in intent_corpus.keys():
    intent_corpus[pair[1]] = sentence

  else : 
    intent_corpus[pair[1]] += sentence

intent_corpus = {a:list(set(b)) for a, b in intent_corpus.items()}

#print(intent_corpus)

In [252]:
intents_vocabulary = []
for intent in intent_corpus:
    for word in intent_corpus[intent]:
        if word not in intents_vocabulary:
            intents_vocabulary.append(word)

intents_bag_of_words = {}
for intent in intent_corpus:
    intents_bag_of_words[intent] = np.zeros(len(intents_vocabulary))
    for stem in intent_corpus[intent]:
        index = intents_vocabulary.index(stem)
        intents_bag_of_words[intent][index] += 1

logfreq_intents_bow = {}
for intent in intents_bag_of_words:
    logfreq_intents_bow[intent] = logfreq_weighting(intents_bag_of_words[intent])
    #print(f'{intent} bag-of-word (logfreq weighted): {logfreq_intents_bow[intent]}')

## User Input Preprocessing Function

In [253]:
def lemmatize_user_query(query):
  # Tokenize
  user_token_query = tokenizer.tokenize(query)

  # Lower casing
  user_token_query  = [word.lower() for word in user_token_query]

  # Remove stopwords and lower casing
  filtered_query = [word for word in user_token_query if word not in english_stopwords]

  # POS-tagging
  filtered_query = nltk.pos_tag(filtered_query , tagset ='universal')

  wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), filtered_query)

  # lemmetize text
  lemmatized_user_query = []
      
  for word, tag in wordnet_tagged:
      if tag is None:
          lemmatized_user_query.append(word)
      else:        
          lemmatized_user_query.append(lemmatiser.lemmatize(word, tag))

  lemmatized_user_query = ' '.join(str(word) for word in lemmatized_user_query)
  
  return lemmatized_user_query


## Chatbot
**code referenced from 'https://medium.com/predict/create-your-chatbot-using-python-nltk-761cd0aeaed3'**

In [254]:
# Set greeting functions
greetings_list = ('hey', 'greetings', 'hi', 'hello', 'bonjour', 'heya', 'hellow', 'howdy', 'yo', 'ahoy')
greetings_response = ['Hello, how may I help you today ?', 'Hi there !', 'Greetings', ]
def greeting(sent):
   for word in sent.split():
      if word.lower() in greetings_list:
         return random.choice(greetings_response)

In [255]:
# define function for responding to user (answer dataset question || failure to understand)
def info_response(user_response):
    bot_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer = None, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf<=0.15):
        bot_response += "Sorry, I don't quite understand you."
        return bot_response
    else:
        bot_response += sentence_tokens[idx]
        return bot_response

In [256]:
# Function to get current time
def time_response():
    date = dt.datetime.now()
    hour = date.strftime("%H")
    minute = date.strftime("%M")
    second = date.strftime("%S")

    return('The time now is %s:%s:%s.' % (hour, minute, second))

In [257]:
# Function to identify similarity to small talk intents
def intents_matching(lemmatized_user_query):
  #Create Bag-of-words for query
  vector_query = np.zeros(len(intents_vocabulary))
  for word in nltk.word_tokenize(lemmatized_user_query):
      if word in intents_vocabulary:
          index = intents_vocabulary.index(word)
          vector_query[index] += 1
  logfreq_vector_query = logfreq_weighting(vector_query)

  identified_intent = []

  for intent in intents_bag_of_words.keys():
    similarity = sim_cosine(logfreq_intents_bow[intent], logfreq_vector_query)

    if (similarity >= 0.2 and similarity != 1) :

      if (not identified_intent):
          identified_intent.append(intent)
          identified_intent.append(similarity)

      if (similarity > identified_intent[1]):
          identified_intent = []
          identified_intent.append(intent)
          identified_intent.append(similarity)

  return identified_intent

In [258]:
flag=True
username = ''
print("Bot: Hello. I am a simple chatbot. What would you like to talk about ? If you wish to exit, type 'Bye'.")
while(flag==True):
    original_user_response = input()
    user_response = original_user_response.lower()

    intent_match = intents_matching(lemmatize_user_query(user_response))

    if(user_response !='bye'):
        if(user_response =='thanks' or user_response=='thank you' ):
            flag=False
            print("Bot: You are welcome..")

        else:
          # greeting intent
            if(greeting(user_response)!=None):
              name_detection = nltk.word_tokenize(original_user_response)
              name_detection = nltk.pos_tag(name_detection)
              chunks = nltk.ne_chunk(name_detection)
              for subtree in chunks.subtrees():
                if subtree.label() == "PERSON":
                    for leaf in subtree.leaves():
                        username = leaf[0]
                        break

              if (username != ''):
                print("Bot: "+greeting(user_response), username,'!')

              else:                
                print("Bot: "+greeting(user_response))

            # intent matched
            elif(intent_match):

              intents = json.loads(open('intents.json').read())
              responses = []

              for intent in intents['intents']:
                  if (intent_match[0] == intent['tag']):
                    if intent_match[0] == 'time':
                      responses.append(time_response())
                      break
                    for response in intent['responses']:
                      responses.append(response)
                    
              print('Bot:', random.choice(responses))

            # information retrieval
            else:
                user_response= lemmatize_user_query(user_response)
                answer = info_response(user_response)
                print("Bot: " + answer)
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Bot: Bye! See you again.")

Bot: Hello. I am a simple chatbot. What would you like to talk about ? If you wish to exit, type 'Bye'.
Hello
Bot: Greetings
Hey I'm Shane
Bot: Greetings Shane !
Hi
Bot: Hello, how may I help you today ? Shane !
What kind of music do you like ?
Bot: I like J-Rock
What time is it ?
Bot: The time now is 13:51:57.
how much is 1 tablespoon of water
Bot: This tablespoon has a capacity of about 15 mL.
That's not correct
Bot: He formulated the Periodic Law, created his own version of the periodic table of elements , and used it to correct the properties of some already discovered elements and also to predict the properties of elements yet to be discovered!
Thank you
Bot: You are welcome..
